<a href="https://colab.research.google.com/github/greyhound101/shopee/blob/main/scoring_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1
def get_neighbors(df, embeddings, KNN = 50, image = True):
    model = NearestNeighbors(n_neighbors = KNN)
    model.fit(embeddings)
    distances, indices = model.kneighbors(embeddings)
    
    # Iterate through different thresholds to maximize cv, run this in interactive mode, then replace else clause with a solid threshold
    thresholds = list(np.arange(4.0, 6.0, 0.1))
    scores = []
    for threshold in thresholds:
            predictions = []
            for k in range(embeddings.shape[0]):
                idx = np.where(distances[k,] < threshold)[0]
                ids = indices[k,idx]
                posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
                predictions.append(posting_ids)
            df['pred_matches'] = predictions
            df['f1'] = f1_score(df['matches'], df['pred_matches'])
            score = df['f1'].mean()
            print(f'Our f1 score for threshold {threshold} is {score}')
            scores.append(score)
    thresholds_scores = pd.DataFrame({'thresholds': thresholds, 'scores': scores})
    max_score = thresholds_scores[thresholds_scores['scores'] == thresholds_scores['scores'].max()]
    best_threshold = max_score['thresholds'].values[0]
    best_score = max_score['scores'].values[0]
    print(f'Our best score is {best_score} and has a threshold {best_threshold}')
    del model, distances, indices
    gc.collect()

class check(Callback):
    def __init__(self, X_seq):
        super(Callback, self).__init__()

        self.X_seq = X_seq
    
    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict_generator(self.X_seq, steps=len(self.X_seq), 
                                                            use_multiprocessing=False, workers=1, 
                                                            max_queue_size=2*4)
            get_neighbors(self.df,y_pred)
            
kappa_metric = KappaEvaluationSeq(val_seq, Y, Y2, smpls, 'val')

In [ ]:
from tensorflow.keras.callbacks import *
from sklearn.neighbors import *
from sklearn.preprocessing import *
class stocasticensembling(Callback):
  def __init__(self):
    #save_se_weights: save after each epoch ?

    self.lr_start   = 0.000001
    self.lr_max     = 0.000005 * BATCH_SIZE
    self.lr_min     = 0.000001
    self.lr_ramp_ep = 10
    self.lr_sus_ep  = 0
    self.model_count = 0
    self.clr_iterations = 0
    self.lr_decay   = 0.8
    self.cycle_len=2
    self.swa_cycle_start_inx=45
    self.iter_per_epoch=107
    self.alpha1=alpha1=self.lr_min
    self.iter_per_cycle = self.cycle_len * self.iter_per_epoch
    self.alpha2=alpha2=self.lr_min*10
    self.cycle_num=0

  def on_train_end(self,logs={}):
    self.weight_update()
    self.model.set_weights(self.swa_weights)
    
  
  
  def on_epoch_end(self,epoch,logs=None):
    self.cycle_num+=1
    if (self._t_cycle() !=1) or (epoch<self.swa_cycle_start_inx):
      return
    self.weight_update()
    self.model_count+=1
  
  
  def on_batch_begin(self,batch,logs=None):
    if self.cycle_num<self.swa_cycle_start_inx:
      self.clr_iterations+=1
      lr=self._clr_schedule()
      K.set_value(self.model.optimizer.lr,lr)
  
  
  
  def weight_update(self):
    weights=self.model.get_weights()
    if self.model_count==0:
      self.swa_weights=weights
    for i in range(0,len(weights)):
      self.swa_weights[i]=(self.swa_weights[i]*self.model_count+weights[i])/(self.model_count+1)
  
  
  def _t_cycle(self):
        return (((self.clr_iterations - 1) % self.iter_per_cycle) + 1) / self.iter_per_cycle
  
  
  def _clr_schedule(self):
    if self.cycle_num>self.swa_cycle_start_inx:
      return ((1.0 - 1.0 *self._t_cycle()) * self.alpha2) + (1.0 *self._t_cycle() *self.alpha1)
    else:
      if self.cycle_num < self.lr_ramp_ep:
            lr = (self.lr_max - self.lr_start) / self.lr_ramp_ep * self.cycle_num + self.lr_start   
      elif self.cycle_num < self.lr_ramp_ep + self.lr_sus_ep:
            lr = self.lr_max    
      else:
            lr = (self.lr_max - self.lr_min) * self.lr_decay**(self.cycle_num - self.lr_ramp_ep - self.lr_sus_ep) + self.lr_min   
      return lr

# Arcmarginproduct class keras layer
class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.reshape(y,(-1,1))
        y = tf.cast(y, dtype=tf.int32)
#         m = tf.reshape(dm,(-1,1))
#         self.m = m
#         self.cos_m = tf.math.cos(m)
#         self.sin_m = tf.math.sin(m)
#         self.th = tf.math.cos(math.pi - m)
#         self.mm = tf.math.sin(math.pi - m) * m
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        one_hot=one_hot[:,0,:]
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        outpu = tf.reshape(output,(-1,N_CLASSES))
        return output

class GeMPoolingLayer(tf.keras.layers.Layer):
    def __init__(self, p=1., train_p=False):
        super().__init__()
        self.p = tf.Variable(p, dtype=tf.float32)
        self.eps = 1e-6

    def call(self, inputs: tf.Tensor, **kwargs):
        inputs = tf.clip_by_value(inputs, clip_value_min=1e-6, clip_value_max=tf.reduce_max(inputs))
        inputs = tf.pow(inputs, self.p)
        inputs = tf.reduce_mean(inputs, axis=[1, 2], keepdims=False)
        inputs = tf.pow(inputs, 1./self.p)
        return inputs

# Function to create our EfficientNetB3 model
def get_model():

    with strategy.scope():

        margin = ArcMarginProduct(
            n_classes = N_CLASSES, 
            s = 30, 
            m = 0.5, 
            name='head/arc_margin', 
            dtype='float32'
            )

        inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
        label = tf.keras.layers.Input(shape = (), name = 'inp2')
        x = efn.EfficientNetB2(weights = 'imagenet', include_top = False)(inp)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
#         x = tf.keras.layers.Dropout(0.25)(x)
#         x = tf.keras.layers.Dense(512)(x)
#         x = tf.keras.layers.BatchNormalization()(x)
#         x = tf.keras.layers.PReLU()(x)
        x = margin([x, label])
        
        output = tf.keras.layers.Softmax(dtype='float32')(x)

        model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])

        opt = tf.keras.optimizers.Adam(learning_rate = LR)

        model.compile(
            optimizer = opt,
            loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
            metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
            ) 
        
        return model
def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1
def get_neighbors(df, embeddings, KNN = 50, image = True):
    model = NearestNeighbors(n_neighbors = KNN)
    model.fit(embeddings)
    distances, indices = model.kneighbors(embeddings)
    
    # Iterate through different thresholds to maximize cv, run this in interactive mode, then replace else clause with a solid threshold
    thresholds = list(np.arange(4.0, 6.0, 0.1))
    scores = []
    for threshold in thresholds:
            predictions = []
            for k in range(embeddings.shape[0]):
                idx = np.where(distances[k,] < threshold)[0]
                ids = indices[k,idx]
                posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
                predictions.append(posting_ids)
            df['pred_matches'] = predictions
            df['f1'] = f1_score(df['matches'], df['pred_matches'])
            score = df['f1'].mean()
            print(f'Our f1 score for threshold {threshold} is {score}')
            scores.append(score)
    thresholds_scores = pd.DataFrame({'thresholds': thresholds, 'scores': scores})
    max_score = thresholds_scores[thresholds_scores['scores'] == thresholds_scores['scores'].max()]
    best_threshold = max_score['thresholds'].values[0]
    best_score = max_score['scores'].values[0]
    print(f'Our best score is {best_score} and has a threshold {best_threshold}')
    del model, distances, indices
    gc.collect()

class check(Callback):
    def __init__(self, X_seq,df,interval):
        super(Callback, self).__init__()
        self.df=df
        self.X_seq = X_seq
        self.interval=interval
    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict_generator(self.X_seq)[:self.df.shape[0]]
            get_neighbors(self.df,y_pred)
            
def train_and_evaluate(df,a):
    train=pd.read_csv('../input/shopee-product-matching/train.csv')
    train['label_group']=le.fit_transform(train['label_group'])
    train=train.drop_duplicates('label_group').reset_index(drop=True)
    train=train.loc[a]
    df=df.loc[df['label_group'].isin(list(train['label_group'].values))]
    print(df.shape)
    # Seed everything
    seed_everything(SEED)
    
    print('\n')
    print('-'*50)
    train=TRAINING_FILENAMES
    train_dataset = get_training_dataset(TRAINING_FILENAMES, ordered = False)
    valid_dataset = get_validation_dataset(VALIDATION_FILENAMES)
    STEPS_PER_EPOCH = count_data_items(train) // BATCH_SIZE
    STEPS_PER_EPOCH_all = count_data_items(TRAINING_FILENAMES) // BATCH_SIZE
    
    K.clear_session()
    model = get_model()
    se=stocasticensembling()
    checkpoint = tf.keras.callbacks.ModelCheckpoint(f'EfficientNetB7_{IMAGE_SIZE[0]}_{SEED}.h5', 
                                                    monitor = 'val_loss', 
                                                    save_best_only = True,
                                                    save_weights_only = True, 
                                                    mode = 'min')
    history = model.fit(train_dataset,
                        steps_per_epoch = 1,
                        epochs = 40,
                        callbacks = [stocasticensembling(),check(valid_dataset,df,5)])

    model.save_weights('weights.hdf5')
a=np.load('../input/stratify-counts/test_4.npy')
df=pd.read_csv('../input/shopee-product-matching/train.csv')
tmp = df.groupby(['label_group'])['posting_id'].unique().to_dict()
df['matches'] = df['label_group'].map(tmp)
df['matches'] = df['matches'].apply(lambda x: ' '.join(x))
df['label_group']=le.fit_transform(df['label_group'])
train_and_evaluate(df,a)